## 📋 Table of Contents
* [File Overview](#files)
* [Training File](#train)
* [EEG File Example](#ex_EEG)
* [Spectrogram File Example](#ex_spec)
* [Test and Submission File](#sub)

In [ ]:
# packages

# standard
import numpy as np
import pandas as pd
import time

# plot
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# configs
pd.set_option('display.max_columns', None) # we want to display all columns in this notebook

# aesthetics
default_color_1 = 'darkblue'
default_color_2 = 'darkgreen'
default_color_3 = 'darkred'

<a id='files'></a>
# File Overview

In [ ]:
!ls -l '../input/hms-harmful-brain-activity-classification'

In [ ]:
!ls -l '../input/hms-harmful-brain-activity-classification/example_figures'

In [ ]:
!ls -l '../input/hms-harmful-brain-activity-classification/train_spectrograms'

In [ ]:
!ls -l '../input/hms-harmful-brain-activity-classification/train_eegs'

<a id='train'></a>
# Training File

In [ ]:
df_train = pd.read_csv('../input/hms-harmful-brain-activity-classification/train.csv')
df_train.head()

In [ ]:
# votes columns
features_vote = ['seizure_vote', 'lpd_vote', 'gpd_vote',
                 'lrda_vote', 'grda_vote', 'other_vote']

In [ ]:
# group by eeg_id to get just one row per EEG
df_train_unique_votes = df_train.groupby('eeg_id')[features_vote].mean()

# calc sum of votes for each row
df_train_unique_votes['vote_sum'] = df_train_unique_votes[features_vote].sum(axis=1)

# normalize votes (=> 100% distribution for each row/EEG)
for f in features_vote:
    df_train_unique_votes[f] = df_train_unique_votes[f] / df_train_unique_votes.vote_sum
    
df_train_unique_votes.head(10)

<a id='ex_EEG'></a>
# EEG File Example

In [ ]:
# load an EEG file
df_eeg = pd.read_parquet('../input/hms-harmful-brain-activity-classification/train_eegs/1000913311.parquet')

In [ ]:
# preview
df_eeg.head()

In [ ]:
# show structure
df_eeg.info()

In [ ]:
# basis stats
df_eeg.describe()

In [ ]:
# list of features
features_eeg = ['Fp1', 'F3', 'C3', 'P3', 'F7', 
                'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz',
                'Fp2', 'F4', 'C4', 'P4', 'F8',
                'T4', 'T6', 'O2', 'EKG']

In [ ]:
# plot all features - time series
for f in features_eeg:
    plt.figure(figsize=(12,3))
    plt.plot(df_eeg[f], color=default_color_1)
    plt.title(f)
    plt.grid()
    plt.show()

In [ ]:
# plot all features - distributions
for f in features_eeg:
    plt.figure(figsize=(12,3))
    plt.hist(df_eeg[f], bins=100,
             color=default_color_1)
    plt.title(f)
    plt.grid()
    plt.show()

In [ ]:
cor_eeg = df_eeg[features_eeg].corr(method='pearson')
plt.figure(figsize=(12,8))
sns.heatmap(cor_eeg, annot=True,
            fmt='.2f',
            linecolor='black', linewidths=.5,
            cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title('Correlation - EEG example')
plt.show()

<a id='ex_spec'></a>
# Spectrogram File Example

In [ ]:
# load spectrogram file
df_spec = pd.read_parquet('../input/hms-harmful-brain-activity-classification/train_spectrograms/1000646093.parquet')

In [ ]:
# preview
df_spec.head()

In [ ]:
# show structure
df_spec.info(verbose=True, show_counts=True)

In [ ]:
# basis stats
df_spec.describe()

### Some plots:

In [ ]:
# plotting function
def plot_spec(i_frequency):
    feature_1 = 'LL_' + i_frequency
    feature_2 = 'RL_' + i_frequency
    feature_3 = 'RP_' + i_frequency
    feature_4 = 'LP_' + i_frequency
    plt.figure(figsize=(10,3))
    plt.scatter(df_spec.time, df_spec[feature_1], label='LL')
    plt.scatter(df_spec.time, df_spec[feature_2], label='RL')
    plt.scatter(df_spec.time, df_spec[feature_3], label='RP')
    plt.scatter(df_spec.time, df_spec[feature_4], label='LP')
    plt.legend(loc='upper right')
    plt.title('Frequency=' + i_frequency)
    plt.grid()
    plt.show()

In [ ]:
# plot for a few frequencies
frequencies = ['0.59', '3.91', '10.16', '19.92']
for freq in frequencies:
    plot_spec(freq)

<a id='sub'></a>
# Test and Submission File

In [ ]:
df_test = pd.read_csv('../input/hms-harmful-brain-activity-classification/test.csv')
df_test

In [ ]:
df_sub = pd.read_csv('../input/hms-harmful-brain-activity-classification/sample_submission.csv')
df_sub

#### 

#### Simply use (grouped) means for a first baseline. See also this notebook: https://www.kaggle.com/code/seshurajup/eda-train-csv

In [ ]:
# simply use means for a first baseline
mean_values = df_train_unique_votes[features_vote].mean()

df_sub.seizure_vote = mean_values['seizure_vote']
df_sub.lpd_vote = mean_values['lpd_vote']
df_sub.gpd_vote = mean_values['gpd_vote']
df_sub.lrda_vote = mean_values['lrda_vote']
df_sub.grda_vote = mean_values['grda_vote']
df_sub.other_vote = mean_values['other_vote']

df_sub

In [ ]:
# save submission file
df_sub.to_csv('submission.csv', index=False)